In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense

In [2]:
X = pd.read_pickle('features.pkl')
Y = pd.read_pickle('labels.pkl')

In [3]:
X.head()

,a0,a1,a2,a3,a4,a5
0,0.166694,-0.660271,-0.853314,0.003423,-0.370807,-0.953668
1,-0.359270,-0.027260,0.467691,-0.304570,-0.411950,-0.978076
2,-0.623079,0.668217,-0.387515,-0.935889,0.464365,-0.493203
3,-0.781723,-0.631258,0.384711,-0.425288,0.621360,0.097770
4,0.366593,-0.441562,0.560481,-0.106615,0.428348,0.599748


In [4]:
Y.head()

,Re(alpha1),Im(alpha1),Re(alpha2),Im(alpha2),Re(alpha3),Im(alpha3),Re(alpha4),Im(alpha4),Re(alpha5),Im(alpha5)
0,0.199643,0.0,-0.829522,-0.352858,-0.829522,0.352858,0.535289,-0.889314,0.535289,0.889314
1,-0.663760,0.0,-0.471552,-0.924256,-0.471552,0.924256,0.592840,-0.403187,0.592840,0.403187
2,-0.959416,0.0,0.444017,-1.426444,0.444017,1.426444,0.506456,-0.577479,0.506456,0.577479
3,-7.064939,0.0,-0.646676,0.000000,1.246553,0.000000,0.054878,-1.183596,0.054878,1.183596
4,-1.537332,0.0,-0.104100,-0.896070,-0.104100,0.896070,0.515660,-0.471891,0.515660,0.471891


In [10]:
X_np = X.to_numpy()
Y_np = Y.to_numpy()

DEGREE = X_np.shape[1] - 1

In [11]:
X_train = X_np[:80000]
Y_train = Y_np[:80000]

X_test = X_np[80000:]
Y_test = Y_np[80000:]

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(80000, 6) (80000, 10)
(20000, 6) (20000, 10)


In [15]:
# some dimensionality handling below
# this is because keras expects 3d tensors (a seq of onehot encoded characters) 
# but we have 2d tensors; just a sequence of numbers.


def reshape(array):
    return np.expand_dims(array, -1)
    
    
print(reshape(X_test).shape)  # batchsize, timesteps, input_dim (or "vocab_size")
print(reshape(Y_test).shape) 

(20000, 6, 1)
(20000, 10, 1)


In [17]:
from keras.models import Sequential
from keras.layers import LSTM, RepeatVector, Dense, TimeDistributed


hidden_size = 128
model = Sequential()

# ENCODER PART OF SEQ2SEQ
model.add(LSTM(hidden_size, input_shape=(DEGREE+1, 1)))

# DECODER PART OF SEQ2SEQ
model.add(RepeatVector(DEGREE * 2))  # this determines the length of the output sequence
model.add((LSTM(hidden_size, return_sequences=True)))
model.add(TimeDistributed(Dense(1)))

model.compile(loss='mean_absolute_error',
              optimizer='adam',
              metrics=['mae'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               66560     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 10, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 128)           131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             129       
Total params: 198,273
Trainable params: 198,273
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
BATCH_SIZE = 128
model.fit(reshape(X_train), reshape(Y_train), batch_size=BATCH_SIZE, epochs=10, verbose=1, validation_data=(reshape(X_test), reshape(Y_test)))

Epoch 1/10
625/625 [==============================] - 38s 61ms/step - loss: 0.5787 - mae: 0.5787 - val_loss: 0.6697 - val_mae: 0.6697
Epoch 2/10
625/625 [==============================] - 37s 60ms/step - loss: 0.5627 - mae: 0.5627 - val_loss: 0.6628 - val_mae: 0.6628
Epoch 3/10
625/625 [==============================] - 38s 60ms/step - loss: 0.5504 - mae: 0.5504 - val_loss: 0.6485 - val_mae: 0.6485
Epoch 4/10
625/625 [==============================] - 38s 60ms/step - loss: 0.5399 - mae: 0.5399 - val_loss: 0.6409 - val_mae: 0.6409
Epoch 5/10
625/625 [==============================] - 37s 60ms/step - loss: 0.5338 - mae: 0.5338 - val_loss: 0.6341 - val_mae: 0.6341
Epoch 6/10
625/625 [==============================] - 37s 60ms/step - loss: 0.5263 - mae: 0.5263 - val_loss: 0.6292 - val_mae: 0.6292
Epoch 7/10
625/625 [==============================] - 38s 60ms/step - loss: 0.5205 - mae: 0.5205 - val_loss: 0.6172 - val_mae: 0.6172
Epoch 8/10
625/625 [==============================] - 37s 60ms

In [29]:
y_pred = model.predict(reshape(X_test));

In [43]:
y_pred[2]

array([[-1.4508247 ],
       [-0.05543272],
       [ 0.23768486],
       [ 0.02247994],
       [ 0.8232915 ],
       [ 0.01112509],
       [-0.36242533],
       [-0.8097664 ],
       [-0.37622356],
       [ 0.85973865]], dtype=float32)

In [44]:
reshape(Y_test[2])

array([[-1.39070781],
       [ 0.        ],
       [ 0.27222145],
       [ 0.        ],
       [ 0.76777958],
       [ 0.        ],
       [-0.35437999],
       [-0.81999672],
       [-0.35437999],
       [ 0.81999672]])

In [66]:
x = 500
y_pred[x] - reshape(Y_test[x])

array([[ 0.01400602],
       [-0.02420072],
       [ 0.01544104],
       [-0.09124837],
       [-0.01127341],
       [ 0.02112756],
       [-0.02802363],
       [ 0.00421211],
       [ 0.01762607],
       [ 0.01840613]])